In [85]:
import pandas as pd

df = pd.read_csv("../src/taxipred/data/taxi_trip_pricing.csv")
df.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


##  Trip price calculation
After looking at the columns I tried to work out the calculations for trip price.
I worked out the the trip price equals ```base fare + (trip distance km * per km rate) + (trip duration minutes * per minute rate)```

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [87]:
df.describe().T.drop("count", axis=1)

,mean,std,min,25%,50%,75%,max
Trip_Distance_km,27.070547,19.905300,1.2300,12.63250,25.8300,38.40500,146.067047
Passenger_Count,2.476842,1.102249,1.0000,1.25000,2.0000,3.00000,4.000000
Base_Fare,3.502989,0.870162,2.0100,2.73000,3.5200,4.26000,5.000000
Per_Km_Rate,1.233316,0.429816,0.5000,0.86000,1.2200,1.61000,2.000000
Per_Minute_Rate,0.292916,0.115592,0.1000,0.19000,0.2900,0.39000,0.500000
Trip_Duration_Minutes,62.118116,32.154406,5.0100,35.88250,61.8600,89.05500,119.840000
Trip_Price,56.874773,40.469791,6.1269,33.74265,50.0745,69.09935,332.043689


In [88]:
df.isnull().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [89]:
df[(df["Weather"] == "Snow") & (df["Base_Fare"].isna())]

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
887,12.67,Morning,Weekday,1.0,Low,Snow,NaN,1.96,0.23,35.59,37.2489
901,2.30,Morning,Weekend,1.0,Medium,Snow,NaN,1.54,0.47,38.81,26.4327


In [90]:
df[df["Base_Fare"].isnull()].head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
25,39.47,Afternoon,Weekday,1.0,Low,Clear,NaN,NaN,0.35,7.59,83.6965
43,45.56,Afternoon,Weekday,NaN,Low,Clear,NaN,0.90,0.50,80.80,85.8840
50,48.51,Evening,Weekday,3.0,High,Rain,NaN,1.00,0.48,110.18,104.1764
92,38.28,Afternoon,Weekday,1.0,Medium,Clear,NaN,1.08,0.39,51.33,63.5611


In [91]:
# check total rows where at least 1 column is null
df[df[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].shape

(266, 11)

## Base fare median/mean
Looking at base fare median and mean to see if I should fill nulls with these values

In [92]:
df.groupby(
    ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
)["Base_Fare"].median()

Time_of_Day  Day_of_Week  Traffic_Conditions  Weather
Afternoon    Weekday      High                Clear      3.03
                                              Rain       3.67
                                              Snow       2.93
                          Low                 Clear      3.51
                                              Rain       3.05
                                                         ... 
Night        Weekend      Low                 Clear      4.33
                                              Rain       3.29
                          Medium              Clear      3.80
                                              Rain       4.41
                                              Snow       2.05
Name: Base_Fare, Length: 65, dtype: float64

In [93]:
df.groupby(
    ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
)["Base_Fare"].mean()

Time_of_Day  Day_of_Week  Traffic_Conditions  Weather
Afternoon    Weekday      High                Clear      3.285556
                                              Rain       3.683000
                                              Snow       2.870000
                          Low                 Clear      3.567843
                                              Rain       3.180000
                                                           ...   
Night        Weekend      Low                 Clear      3.901429
                                              Rain       3.150000
                          Medium              Clear      3.683750
                                              Rain       3.870000
                                              Snow       2.050000
Name: Base_Fare, Length: 65, dtype: float64

## Filling nulls
On rows which contain only a single null in the trip price calculation I applied a rule based calculation.
And decided to fill base fare with mean values where the row contain more than a single null for the calculation

In [94]:
rules = {
    "Trip_Price": lambda r: r["Base_Fare"] + (r["Trip_Distance_km"] * r["Per_Km_Rate"]) + (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"]),
    
    "Base_Fare": lambda r: r["Trip_Price"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"]) - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"]),
    
    "Trip_Distance_km": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"])) / r["Per_Km_Rate"],
    
    "Per_Km_Rate": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"])) / r["Trip_Distance_km"],
    
    "Trip_Duration_Minutes": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"])) / r["Per_Minute_Rate"],
    
    "Per_Minute_Rate": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"])) / r["Trip_Duration_Minutes"]   
    
}

In [95]:
df_filled = df.copy()

updated = True
while updated:
    updated = False
    
    for col, func in rules.items():
        mask = df_filled[col].isnull()
        
        if mask.any():
            before = df_filled.loc[mask, col].copy()
            df_filled.loc[mask, col] = df_filled.loc[mask].apply(func, axis=1)
            
            if not df_filled.loc[mask, col].equals(before):
                updated = True
    
    bf_before = df_filled["Base_Fare"].copy()
    df_filled["Base_Fare"] = df_filled["Base_Fare"].fillna(df_filled.groupby(["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
    )["Base_Fare"].transform("mean").round(2))
    
    if not df_filled["Base_Fare"].equals(bf_before):
        updated = True

In [96]:
df_filled.isnull().sum()

Trip_Distance_km          5
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                 0
Per_Km_Rate              10
Per_Minute_Rate           5
Trip_Duration_Minutes    11
Trip_Price               11
dtype: int64

In [97]:
df_filled[df_filled[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].shape

(21, 11)

In [98]:
df_filled[df_filled[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].shape

(21, 11)

## Droping rows
Decided to drop the remaining rows with more than one null in the trip price calculation.
Since the features containing nulls have a greater impact on the trip price than base fare and these rows only makes up 2.1% of the data.
Removing these makes the model results more reliable.

In [99]:
df_clean = df_filled.dropna(subset=["Trip_Distance_km", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"])

In [100]:
df_clean.isnull().sum()

Trip_Distance_km          0
Time_of_Day              50
Day_of_Week              48
Passenger_Count          49
Traffic_Conditions       49
Weather                  48
Base_Fare                 0
Per_Km_Rate               0
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price                0
dtype: int64

## Droping columns
Dropping weather, traffic_conditions and passenger_count since I wont use weather and traffic data, and can add them back later if I have time. And dropping passenger_count since it dont affect the trip price.

In [101]:
df_clean = df_clean.drop(columns=["Passenger_Count", "Traffic_Conditions", "Weather"])
df_clean

,Trip_Distance_km,Time_of_Day,Day_of_Week,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,3.25,0.62,0.43,40.57,50.2009
2,36.87,Evening,Weekend,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,3.48,0.51,0.15,116.81,36.4698
4,8.64,Evening,Weekday,2.93,0.63,0.32,22.64,15.6180
...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,Weekday,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,Weekday,2.67,0.82,0.17,114.94,61.2090


## Filling categorical nulls
Filling time_of_day and day_of_week nulls with 'unknown' so i dont need to drop the rows and lose data for the ml model

In [102]:
df_clean.fillna({"Time_of_Day": "Unknown"}, inplace=True)
df_clean.fillna({"Day_of_Week": "Unknown"}, inplace=True)
df_clean.isnull().sum()

Trip_Distance_km         0
Time_of_Day              0
Day_of_Week              0
Base_Fare                0
Per_Km_Rate              0
Per_Minute_Rate          0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64

## Outliers
Checking which columns containing outliers

In [103]:
import numpy as np
numeric_cols = df_clean.select_dtypes(include=np.number).columns
outlier_flags = pd.DataFrame(index=df.index)


for col in numeric_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_flags[col] = ~df_clean[col].between(lower, upper)

    
outlier_flags.sum()

Trip_Distance_km         19
Base_Fare                 0
Per_Km_Rate               0
Per_Minute_Rate           1
Trip_Duration_Minutes     0
Trip_Price               24
dtype: object

In [ ]:
# Removing outliers
for col in numeric_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

df_clean

,Trip_Distance_km,Time_of_Day,Day_of_Week,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,3.25,0.62,0.43,40.57,50.2009
2,36.87,Evening,Weekend,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,3.48,0.51,0.15,116.81,36.4698
4,8.64,Evening,Weekday,2.93,0.63,0.32,22.64,15.6180
...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,Weekday,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,Weekday,2.67,0.82,0.17,114.94,61.2090


In [105]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder

classification_models = [
    RandomForestClassifier(),
    XGBClassifier(),
    LogisticRegression(),
    KNeighborsClassifier()
]

regression_models = [
    RandomForestRegressor(),
    XGBRegressor(),
    LinearRegression(),
    Ridge(),
    KNeighborsRegressor()
]

def train_eval_model(df: pd.DataFrame, target_column: str):
    df = df[df[target_column].notnull()].dropna()
    X, y = df.drop(target_column, axis=1), df[target_column]
    
    X_encoded = pd.get_dummies(X, drop_first=True)
    
    X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.33, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_score = float("inf")
    best_model = None
    
    for model in regression_models:
        if isinstance(model, KNeighborsRegressor):
            for k in range(1,15):
                model.n_neighbors = k
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
                
                score = root_mean_squared_error(y_test, y_pred)
                
                if score < best_score:
                    best_score = score
                    best_model = model
        else:
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            score = root_mean_squared_error(y_test, y_pred)
            
            if score < best_score:
                best_score = score
                best_model = model
    
    return best_model

